# Case Study: Classification

This dataset is a **classification problem**, concerning whether if the loan was approved or not.

- **Loan_ID** - unique loan ID
- **Gender** - male/female
- **Married** - yes/no
- **Dependents** - number of dependents
- **Education** - graduate/undergraduate
- **Self_Employed** - yes/no
- **ApplicantIncome** - applicant income
- **CoapplicantIncome** - coapplicant income
- **LoanAmount** - loan amount in thousands
- **Loan_Amount_Term** - term of lean in months
- **Credit_History** - credit history meets guidelines
- **Property_Area** - Urban/semi and Rural
- **Loan_Status** - approved yes/no 

We are given 2 set, the training set and the test set.

The **training set** contains 614 samples and 13 features, 12 of which are the independent variables and the last feature `Loan_Status` is the dependent variable.

The **test set** contains 367 samples with the same 12 features but without the `Loan_Status` columns. So it will be representing the unseen data that we will be implementing our model on.

In [ ]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt

In [ ]:
pd.__version__

In [ ]:
np.__version__

In [ ]:
import matplotlib
matplotlib.__version__

## 1. Load the data

In [ ]:
#get the data from google classroom
#kaggle competition
df_train = pd.read_csv("data/train_LoanPrediction.csv")
df_test  = pd.read_csv("data/test_LoanPrediction.csv")

In [ ]:
#1. get the shape (rows, columns)
df_train.shape  #(614 rows, 13 columns)

#2. try .head()
df_train.head()
# df_train.info()

#3. try .describe()
# df_train.describe()

#4. check whether our data class is balanced, using value_counts(normalize=True)
# df_train['Loan_Status'].value_counts() #if you don't put normalize=True, will give you raw number

#our class is imbalanced....
#it means, we have to downsample Y, to equal N

In [ ]:
df_train['ApplicantIncome'].max()

In [ ]:
condMaxAI = df_train['ApplicantIncome'] == 81000
df_train[condMaxAI]

In [ ]:
df_train.loc[409]

In [ ]:
df_train['Loan_Status'].value_counts()

### Balance the class

- We need to balance the label in the training set, because imbalanced class affects the model during training
- If we do not balance the class, you can also try look at the SMOTE (https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html) algorithm which dynamically augments data on the fly.

In [ ]:
#1. set the cond = Y, and cond = N
condY = df_train.Loan_Status == 'Y'  #to 0 or 1
condN = df_train.Loan_Status == 'N'

#2.. df[condY].sample(n = 192)
#    df[condN].sample
#    we do this many times, do whole ML thing, and do the average....
df_trainY = df_train[condY].sample(n=192, random_state=999)
df_trainN = df_train[condN] #also 192

#3. concat these two dfs
df_train = pd.concat([df_trainY, df_trainN])

df_train.Loan_Status.value_counts()


### Label encoding

This is a useful step to do before EDA, so that categories are turned into numbers.


In [ ]:
df_train.dtypes

In [ ]:
#do label encoding the Loan Status
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_train['Loan_Status'] = le.fit_transform(df_train['Loan_Status'])

#very interesting...our testing set has no loan status, because this is actually a real testing set....

print("I want to know the mapping of 0 and 1", le.inverse_transform([0, 1]))

In [ ]:
#label encode education as well
df_train['Education'] = le.fit_transform(df_train['Education'])
df_test['Education']  = le.transform(df_test['Education'])

#optionally you can use pd.map
#df_train.map({'Graduate': 0, 'Not Graduate': 1})

### One hot encoding

In [ ]:
df_train.columns

In [ ]:
df_train.Property_Area.value_counts()

In [ ]:
#why one-hot encode Property Area, not encode it into 0, 1, 2
#if we have more than two categories, if we encode into 0, 1, 2
#we create a unintentional order, i.e., the model "may" think that 0 < 1 < 2

#what if we have like 5000 categories....
    #one hot encode this will result in 5000 columns --> this introduces burden to the machine learning model
    #1. NLP --> we basically do like this....
    #2. General case -->
        #1st:  Group these categories into bigger categories, and then one-hot encode
        #2nd:  (not so good) - do label encoding anyway......but note the possible order effect
        
#one hot encoding
#[1, 0, 0], [0, 1, 0], [0, 0, 1]
#one thing you need to know is that you can always cut down one column
#[1, 0], [0, 1], [0, 0]  ==> this is possible, and will save you one more column...
#also this is recommended, because it will not introduce unwanted correlation.....
#this is done by setting "drop_first=True"

df_train = pd.get_dummies(df_train, columns=['Property_Area'], drop_first=True)
df_test  = pd.get_dummies(df_test,  columns=['Property_Area'], drop_first=True)

In [ ]:
df_train.head()

## 2. EDA (Exploratory Data Analysis)

In [ ]:
#0. Check the data type
df_train.dtypes
df_train.drop(columns = ['Loan_ID'], inplace=True)  #this is not a useful feature....
df_train.columns

In [ ]:
import seaborn as sns

#1. You want to divide into categorical and numerical columns using select_dtypes
df_train.dtypes
num_col = df_train.select_dtypes(include=['int64', 'float64'])
# numcol = numcol.drop([]) #Loan_Status
cat_col = df_train.select_dtypes(exclude=['int64', 'float64'])
num_col.columns, cat_col.columns

### Bar plots

In [ ]:
#2. for numerical type, lets plot some a bar plot with Loan Status
for col in num_col.columns:
    sns.barplot(x = df_train['Loan_Status'], y = df_train[col])
    plt.show()

### Countplot / Histogram

In [ ]:
#3. for categorial type, you may want to use countplot with Loan Status
for col in cat_col.columns:
    sns.countplot(x = df_train[col], hue = df_train['Loan_Status'])
    #similar to hue in scatterplot
    plt.show()
    # sns.countplot()  #because categorical, you need to count, not simply use the magnitude...so cannot use bar...

### Correlation matrix

In [ ]:
sns.heatmap(df_train.corr(), annot=True)  #this is only for numeric values....

### Predictive Power Score

In [ ]:
import ppscore as pps

# before using pps, let's drop country and year
dfcopy = df_train.copy()

#this needs some minor preprocessing because seaborn.heatmap unfortunately does not accept tidy data
matrix_df = pps.matrix(dfcopy)[['x', 'y', 'ppscore']].pivot(columns='x', index='y', values='ppscore')

#plot
plt.figure(figsize = (15,8))
sns.heatmap(matrix_df, vmin=0, vmax=1, cmap="Blues", linewidths=0.5, annot=True)

## 3. Feature Engineering

Skipped

## 4. Feature Selection

Since our dataset is already splitted at the dataset level, thus we do not need to hurry to select the features.  We can preprocess and then select later.

#forward selection - process of starting with one feature, and slowly adding one feature at a time, until the performance does not improve by certain threshold you want.

## 5. Preprocessing

### Null values

In [ ]:
#1. please check the missing value in df_train, and of course df_test
df_train.isna().sum()

#Gender:            majority or ratio
#Dependents:        majority or ratio
#Self_Employed:     majority or ratio
#LoanAmount:        median or mean
#Loan_Amount_Term:  majority or ratio
#Credit_History:    majority or ratio

If we have a missing categorical feature, we can replace with three things:
- mode (most frequent) --> bad
- ratio (keep the ratio intact) --> preferred
- "No category" --> believe replacing is not a good option

In [ ]:
ratio = df_train['Credit_History'].value_counts(normalize=True)
print(ratio)

In [ ]:
ratio.dtypes

In [ ]:
type(ratio)

In [ ]:
# #either replace with the majority or keep the ratio
# #majority is good when the majority takes over like 90% of the population
# #otherwise, ratio is better

# #in this case, it's better to fill the missing value with ratio
# #ratio means that we want to keep the ratio 75 25 AFTER filling....
print("Credit history of 1: ", ratio[1])
print("Credit history of 0: ", ratio[0])
print(ratio.index)


In [ ]:
# #we gonna fill the missing value such that the ratio is kept unchanged
# #format: fillna(pd.Series(random.choice(1, 0), p=probability of the ratios), inplace=True)
missing = df_train.Credit_History.isna().sum()

num1 = int(np.round(ratio[1] * missing))
num0 = int(np.round(ratio[0] * missing))

print(num1, num0)


In [ ]:
df_train['Credit_History'].fillna(1.0, limit = num1, inplace=True)
df_train['Credit_History'].fillna(0.0, limit = num0, inplace=True)

#pd.series is ok, but i feel like limit is much easier

In [ ]:
df_train['Credit_History'].isna().sum()

In [ ]:
#fill the missing for the test set for credit history

#number of missing values
missing = df_test['Credit_History'].isna().sum()

#get the row index where the missing values located at
cond = df_test['Credit_History'].isna()  #list of true and false
missing_index = df_test[cond].index

#get ratio
ratio = df_test['Credit_History'].value_counts(normalize=True)
print("Test ratio: ", ratio)

series = pd.Series(np.random.choice((1, 0), p=[ratio[1], ratio[0]], size=missing), index=missing_index)
print(series.value_counts(normalize=True))

print("Before: ", df_test['Credit_History'].isna().sum())
print("Before: ", df_test['Credit_History'].value_counts(normalize=True))

df_test['Credit_History'].fillna(series, inplace=True)
df_test['Credit_History'].isna().sum()

print("After: ", df_test['Credit_History'].isna().sum())
print("After: ", df_test['Credit_History'].value_counts(normalize=True))

In [ ]:
sns.displot(df_train.LoanAmount)
print(f"median: {df_train.LoanAmount.median()}; mean: {df_train.LoanAmount.mean()}")

#can you guys help me fill in the LoanAmount with median()
df_train['LoanAmount'].fillna(df_train['LoanAmount'].median(), inplace=True)

df_train.isna().sum()

Here I stop because I am too lazy as the rest of the features won't be used for training.

In [ ]:
df_test.isna().sum()

### Scaling

In [ ]:
#StandardScaler is usually used for continuous values
#here, Education, Credit_History, and Property Area are all categorical
#standardizing them will lose the category, so we will not do!!!

## 6. Modeling

In [ ]:
#1. set the X and y
X_train = df_train[ ['Education', 'Credit_History', 'Property_Area_Semiurban', 'Property_Area_Urban'] ]
y_train = df_train['Loan_Status']

X_test = df_test[ ['Education', 'Credit_History', 'Property_Area_Semiurban', 'Property_Area_Urban'] ]
#no y_test, because this dataset does not have the answer....

#2. make sure the shape of X is (samples, features), and y is (samples, )
assert X_train.ndim == 2
assert X_test.ndim  == 2
assert y_train.ndim == 1

print("(samples, features): ", X_train.shape)
print("(samples,         ): ", y_train.shape)
print("(samples, features): ", X_test.shape)


In [ ]:
#3. perform cross validation with a selected set of models, to scope down which model is among the best

#3.1 specify the list of algorithms
#this is for classification
##Baselines
from sklearn.linear_model import LogisticRegression  #drawing a line based on linear regression but used for classification
from sklearn.naive_bayes import GaussianNB  #drawing a line based on probability

##Situational (but we don't use much)
from sklearn.neighbors import KNeighborsClassifier

##Complex
from sklearn.ensemble import RandomForestClassifier  #using trees to classify
from sklearn.svm import SVC  #drawing a line based on maximum distance
from sklearn.ensemble import GradientBoostingClassifier  #<<-------is the MOST complex
    #XGBoost but is not in sklearn, import XGBoost #<---most powerful algorithm for tabular data (not images, signal)

#any random_state you can use......up to you 
lr = LogisticRegression(random_state=999)
rf = RandomForestClassifier(random_state=999)
sv = SVC(random_state=999)

models = [lr, rf, sv]

#3.2 perform cross validation using KFold
from sklearn.model_selection import KFold, cross_val_score

kfold = KFold(n_splits = 5, shuffle = True, random_state=999)

for model in models:
    score = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')  #f1, recall, precision, accuracy
    print("Scores: ", score, "- Scores mean: ", score.mean(), "- Scores std (lower better): ", score.std())  #out of 1 ; 1 means perfect accuracy
    #lr, rf, sv

#when they are very close, 
    #how we choose?  we choose the simplest model...
    #what does simplest mean?  LogisticRegression is simplest, and SVM is the most complex....
    
#overfitting vs. underfitting
#overfitting means your model learns too much about the training set, 
    #so it cannot generalize to validation set or testing set (unseen data)
    #is NOT good - it means your model CANNOT GENERALIZE to unseen data (validation or test set)
    
#underfitting means your model just fail to see any patterns in the training set
    #this is easy: just look at score

In [ ]:
#4. grid search to find the best version of that model
#remind you: grid search is BASICALLY same as cross validation but for the same model

from sklearn.model_selection import GridSearchCV

model = LogisticRegression(random_state=999)  #<----this is the model I choose, after cross validation

param_grid = dict()
param_grid['solver'] = ['newton-cg', 'lbfgs', 'liblinear']  #this is listed in the sklearn website

#add more parameters here
#param_grid[parameter] = list of parameters to search

#refit means it will pick the best model, and fit again, so it means grid is already the best model after this line
grid = GridSearchCV(model, param_grid, scoring="accuracy", cv=kfold, refit=True, return_train_score=True)
#scoring = f1, recall, precision, accuracy

#fit the grid, which will basically do cross validation across all combinatiosn, here we only have 3 comb
grid.fit(X_train, y_train)  #remember to use only training set here....

#print the best parameters and accuracy
# print(grid.best_params_)
# print(grid.best_score_)
print(grid.cv_results_)

#this score is cross-validation score, basically the accuracy/precision/etc on the validation set


### Classification metrics

Let us study some classification metrics that are quite different from the $r^2$ or mean squared error that we see from the regression.  Let me define a confusion matrix that looks like this:

<code>
		 	    Actual
			    +	   -
Predicted +     TP     FP
          -     FN     TN
</code>

TP is defined as true positives, FP as false positives, FN as false negatives, and TN as true negatives.

#### Accuracy, Recall, Precision, F1

Accuracy is straightforward

$$ \text{Accuracy} = \frac{\text{TP} + \text{TN}}{\text{TP} + \text{TN} + \text{FP} + \text{FN}} $$

**Accuracy** is mostly avoided, unless your model is really balanced of both positives and negatives.  Instead, more useful classification metrics would be precision, recall, and f1-score

$$ \text{Precision} = \text{TP} / (\text{TP} + \text{FP}) $$

**Precision** is useful as metric when you want to prioritize removing false positive.  Example is search engine in which you do not want to return any search results that are "false positive"

$$ \text{Recall} = \text{TP} / (\text{TP} + \text{FN}) $$

**Recall** is useful as metric when you want to prioritize removing false negative.  Example is cancer detection in which you do not want to miss detecting any real positive (i.e., false negative).

$$ \text{F1} = 2 x \frac{\text{Precision} * \text{Recall}}{\text{Precision} + \text{Recall}} $$

**F1** is simply seeking a balance between Precision and Recall.  Also F1 is good metric when there is an uneven class distribution (large number of actual negatives)

sklearn provides a easy-to-use <code>**sklearn.metrics.classification_report**.</code> API that report these four metrics:

In [ ]:
#confusion matrix 
'''
T = True, F = False
P = Positive, N = Negative
'''

''' 
                        Actual
                    P               N
             P     TP              FP
predict   
             N     FN              TN
'''

#0 here is negative
#1 here is positive

y     = [0, 0, 1, 1, 0, 1]
ypred = [1, 0, 1, 1, 0, 1]

#accuracy = (TP + TN) / (TP + FP + FN + TN)
#accuracy = 3 + 2  /  6 = 5 / 6 = 83%

#but accuracy is NOT good if your data is imbalanced (see actual y)

#recall, precision, f1-score
#recall = TP  / (TP + FN)
#recall = 3   / (3  + 0) = 100%  Recall is used when you don't like FN, e.g., critical - cancer, zombie, terrorists

#precision = TP / (TP + FP)
#precision = 3 /  (3 + 1) = 75% Precision is used when you don't like FP, e.g., search engine

#f1-score = 2 * ( (recall * precision) / (precision + recall) )
        #when precision and recall are both important.....
        #   2 * ( (1      * 0.75)     /   (0.75     +    1  ) )
        
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#if you want all in one table
from sklearn.metrics import classification_report

#if you want the confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

#format (ytrue, ypred)
print(accuracy_score(y, ypred))
print(precision_score(y, ypred))
print(recall_score(y, ypred))
print(f1_score(y, ypred))

print(classification_report(y, ypred))

cm = confusion_matrix(y, ypred)
cmp = ConfusionMatrixDisplay(cm, display_labels=[0, 1])

cmp.plot()

#for regression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


Let's explain the classification report a bit:

- <code>weight_avg</code> mutiplies score with the number of true labels in that class, favoring the majority label
#macro simply average, thus better for imbalanced dataset

- <code>macro_avg</code> 

- <code>support</code> refers to number of samples belonging to that class

Between precision or recall or f1?
- Use **recall** for dangerous stuffs - healthcare, security, fraud. You don't care about false positives too much (i.e., false alarm) because you want to always stay fully cautious.

- Use **precision** for search/identifying something - HR, search engine, etc.  You care a lot about false positive, i.e., you want to identify really good sample.

- Use **f1-score** - basically a good bet for almost all imbalanced dataset.

Final recommendations:
- in balanced dataset, just simply use <code>accuracy</code>
- in imbalanced dataset, use <code>macro avg f1-score</code> or <code>macro precision</code> or <code>macro recall</code>

## 7. Testing

In [ ]:
pred_y = grid.predict(X_test)

print(pred_y)

#if we have y_test
#precision_score(y_test, pred_y)

#three cases (assume I have y_test):
    #if I got score much lower than 0.7, e.g., 0.5 0.4, i have serious overfitting issue
    #if I got score near 0.7 or just a little lower, or just a little higher, like 0.65 0.6 0.75, i think its ok
    #if I got score like much much higher than 0.7, like 0.95, something is wrong.....maybe your test set is broken or something
        #but this case rarely happens!
        
#overfitting means your model CANNOT GENERALIZE to unseen data....your unseen data can be validation or test set....

Since I don't really have the ground truth, I don't know the goodness of my model.  But let's assume I make some fake ground truth:

In [ ]:
fake_y_test = np.random.randint(0, 2, size = (pred_y.shape))

In [ ]:
accuracy_score(fake_y_test, pred_y)  #precision_score, recall_score, classification_report, confusion_matrix

## 8. Analysis: Feature Importance

- Basically understanding which features are important for prediction
- Different algorithms have different way for feature importance
- For Logistic Regression, similar to Linear Regression, you can look at the coeffients/weights/slope

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_estimator_.coef_
#['Education', 'Credit_History', 'Property_Area_Semiurban', 'Property_Area_Urban']

#predict LoanStatus, LoanAmount

In [ ]:
feature_importance = pd.DataFrame(['Education', 'Credit_History', 'Property_Area_Semiurban', 'Property_Area_Urban'], 
                                  columns=['features'])
feature_importance["importance"] = grid.best_estimator_.coef_[0]

feature_importance = feature_importance.sort_values(by = ['importance'], ascending=True)

feature_importance.plot.barh(x='features', y='importance')

## 9. Inference

In [ ]:
import pickle

# save the model
filename = 'model/Loan_Prediction.pkl' # pkl does not matter, you can do .everything
pickle.dump(grid, open(filename,'wb'))

# Load the model
loaded_grid=pickle.load(open(filename,'rb'))

# if you have new data, then you fit again....but using loaded_grid
# which is a process of training more.....once you have more data....

# or another way is
# put all the dataset together, and train like it is new
    #this is possible ONLY if your dataset is not that big......

In [ ]:
#let's try to create one silly example
df_train[['Education', 'Credit_History', 'Property_Area_Semiurban', 'Property_Area_Urban', 'Loan_Status']].loc[1]

In [ ]:
sample = np.array([[0, 1, 0, 0]])

In [ ]:
predicted_loan_status= loaded_grid.predict(sample)
predicted_loan_status

It's wrong!  So certainly we need to improve the model :-)